## Advent of Code - Day 5

In [1]:
import tempfile
from contextlib import contextmanager

In [2]:
@contextmanager
def test_file(test_input):
    with tempfile.NamedTemporaryFile('r+') as f:
        f.write(test_input)
        f.seek(0)
        yield f

### Part 1

Idea: Scan through the input and push each character to a stack. When the top of the stack and the next character read have the same type but opposite polarity, discard both. This is $O(n)$ where $n$ is the length of the input. 

Start by creating a generator for each unit in the polymer:

In [167]:
def get_polymer(path):
    """Yields each unit from the polymer stored at path."""
    chunk_size = 1024
    with open(path, 'r') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            for unit in chunk:
                if unit == '\n':
                    break
                yield unit

Check that this works:

In [168]:
test_input = "dabAcCaCBAcCcaDA"

with test_file(test_input) as f:
    print(list(get_polymer(f.name)))

['d', 'a', 'b', 'A', 'c', 'C', 'a', 'C', 'B', 'A', 'c', 'C', 'c', 'a', 'D', 'A']


We'll need a function that checks if two units react:

In [169]:
def react(unit_a, unit_b):
    """Returns true if the two units react."""
    return unit_a.lower() == unit_b.lower() and unit_a != unit_b

In [170]:
react('a', 'A')

True

Now to iterate over the units, pushing each to a stack. If the top of the stack react, discard both and continue:

In [171]:
def reaction_result(polymer, react=react):
    """Returns the result after reacting the polymer."""
    stack = []
    for unit in polymer:
        if stack and react(unit, stack[-1]):
            stack.pop()
            continue
        stack.append(unit)
    return ''.join(stack)

Check that this works for the example input:

In [172]:
reaction_result(test_input) == "dabCBAcaDA"

True

Run on the real input:

In [173]:
!ls

day_5.ipynb  input


In [174]:
print(len(reaction_result(get_polymer('input'))))

10774


### Part 2

Naive solution: Iterate over unique unit type, filter units of that type and compute the lengths ensuring to keep a record of the shortest polymer and the unit removed.

In [175]:
def remove(unit, polymer):
    """Returns an iterator over polymer with both types of unit removed."""
    return filter(lambda u: u.lower() != unit.lower(), polymer)

In [179]:
def shortest_polymer(polymer):
    """Returns len of shortest polymer after removal of all instances of a unit."""
    units = {c.lower() for c in polymer}
    
    shortest_len = None
    for unit in units:
        collapsed_len = len(reaction_result(remove(unit, polymer)))
        if shortest_len is None or collapsed_len < shortest_len:
            shortest_len = collapsed_len
            
    return shortest_len

Check that this works for the test input:

In [180]:
shortest_polymer(list(test_input))

4

Run on the real input - note it's OK to react the polymer first to speed things up a little!

In [182]:
%%timeit
shortest_polymer(reaction_result(get_polymer('input')))

199 ms ± 2.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
